In [1]:
from Zaif_api import Zaif_api
# import time
import datetime

In [2]:
############### Parameters ############################################################################
# How much to buy (in yen)
budget = 400

# API key
api_key = ""

# API secret
api_secret = ""

#######################################################################################################

In [3]:
##### Define global_nonce #####
# Zaif only allows limited trading api calls per second
global_nonce = int(time.time())

In [4]:
api = Zaif_api.zaif(api_key, api_secret)

# テスト用

api.cancel_all_orders()

In [5]:
balance_amount, balance_available = api.my_balance("jpy")
display(balance_amount, balance_available)

NameError: name 'global_nonce' is not defined